In [1]:
from datetime import datetime, date
import os

import pandas as pd
import pyspark 
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.pandas as ps

spark = SparkSession.builder.appName("Practice").getOrCreate()

22/06/11 13:49:15 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
22/06/11 13:49:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/11 13:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
df1 = spark.read.csv('dummy_schedule_data.csv', header=True, sep=",")
df2 = spark.read.csv('dummy_schedule_data2.csv', header=True, sep=",")

In [27]:
df_union = df1.union(df2)

In [28]:
df_union.show(truncate=False)

+----------+---------------------+---------------------+-----------------+-----------+
|start_time|away_team            |home_team            |date             |proper_date|
+----------+---------------------+---------------------+-----------------+-----------+
|7:00p     |Dallas Mavericks     |Golden State Warriors|Tue, May 16, 2022|2022-05-16 |
|7:00p     |Golden State Warriors|Dallas Mavericks     |Tue, May 16, 2022|2022-05-16 |
|7:30p     |Dallas Mavericks     |Golden State Warriors|Tue, May 16, 2022|2022-05-16 |
|7:30p     |Golden State Warriors|Dallas Mavericks     |Tue, May 16, 2022|2022-05-16 |
|7:30p     |Miami Heat           |Boston Celtics       |Tue, May 23, 2022|2022-05-23 |
|8:30p     |FakeGame1            |FakeGame2            |Tue, May 23, 2022|2022-05-23 |
+----------+---------------------+---------------------+-----------------+-----------+



In [29]:
df_union.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- date: string (nullable = true)
 |-- proper_date: string (nullable = true)



In [30]:
# these are all the same

df_union.filter("start_time == '7:30p'").show()

df_union.filter(df_union.start_time == '7:30p').show()

df_union.where(df_union.start_time == '7:30p').show()

df_union.where(df_union['start_time'] == '7:30p').show()

df_union.where(df_union['start_time'] == F.lit('7:30p')).show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [51]:
groupby1 = df_union.groupBy(df_union.start_time).count()

print(groupby1.head(5))

groupby2 = df_union \
    .filter(df_union.start_time == '7:30p') \
    .groupBy(df_union.start_time) \
    .count()

print(groupby2.head(5))

groupby_df = groupby2.toPandas()

[Row(start_time='7:00p', count=2), Row(start_time='7:30p', count=3), Row(start_time='8:30p', count=1)]
[Row(start_time='7:30p', count=3)]


In [44]:
df_union.where(df_union['start_time'] == '7:30p').show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [45]:
df_union.where(df_union['start_time'] == F.lit('7:30p')).show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [52]:
df_union.createOrReplaceTempView("test_table")
df = spark.table("test_table")

In [60]:
dd = spark.sql("select * from test_table where start_time = '7:30p';").toPandas() # pulls the data into a pandas a dataframe
dd2 = spark.sql("select * from test_table where start_time = '7:30p';").collect() # pulls the data into a nested python list
spark.sql("select * from test_table where start_time = '7:30p';").show() # prints it out into the notebook

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [69]:
# being able to read plans helps you to predict potential bottlenecks in your larger queries.
# spark keeps a whole bunch of info about this test_table, which came from df_union, which came from 2 separate csv files i loaded in.
# project means the columns that will be projected (or selected).
# hashaggregate means data aggregation (with group by).
# this prints out the parsed, analyzed, and optimized logical plans and then the physical plan.

# the physical plan is what gets ran on the executors
spark.sql("select start_time, count(*) as num_records from test_table where start_time = '7:30p' group by start_time;").explain(extended = True)

== Parsed Logical Plan ==
'Aggregate ['start_time], ['start_time, 'count(1) AS num_records#1198]
+- 'Filter ('start_time = 7:30p)
   +- 'UnresolvedRelation [test_table], [], false

== Analyzed Logical Plan ==
start_time: string, num_records: bigint
Aggregate [start_time#286], [start_time#286, count(1) AS num_records#1198L]
+- Filter (start_time#286 = 7:30p)
   +- SubqueryAlias test_table
      +- View (`test_table`, [start_time#286,away_team#287,home_team#288,date#289,proper_date#290])
         +- Union false, false
            :- Relation [start_time#286,away_team#287,home_team#288,date#289,proper_date#290] csv
            +- Relation [start_time#312,away_team#313,home_team#314,date#315,proper_date#316] csv

== Optimized Logical Plan ==
Aggregate [start_time#286], [start_time#286, count(1) AS num_records#1198L]
+- Union false, false
   :- Project [start_time#286]
   :  +- Filter (isnotnull(start_time#286) AND (start_time#286 = 7:30p))
   :     +- Relation [start_time#286,away_team#287